## Preambule

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as pth
import xarray as xr
from plotly.subplots import make_subplots
from tqdm import tqdm
import plotly.graph_objects as go
from pathlib import Path
import yaml

## Read

In [27]:
with open("../Configuration/config.yaml", "r") as stream:
    settings = yaml.load(stream, Loader=yaml.Loader)
integers_touse = [0, 1, 3, 4, 5, 7, 8, 9]
list_of_models = np.array([settings['models'][m]['full_name'] for m in settings['models'].keys()])[integers_touse]
list_of_model_shortnames = np.array([settings['models'][m]['short_name'] for m in settings['models'].keys()])[integers_touse]
list_of_colors = np.array([settings['models'][m]['color'] for m in settings['models'].keys()])[integers_touse]
ind_xr = xr.open_dataset(settings['paths']['data']+"XRindicators.nc").sel(Model=list_of_models, Region='Europe')
data_xr = xr.open_dataset(settings['paths']['data']+"XRdata.nc").sel(Model=list_of_models, Region='Europe')

## Check coverage

In [28]:
ar = np.array(data_xr.sel(Variable="Emissions|CO2", Time=2050, Scenario=settings['scenarios_c400']+['DIAG-NPI']).Value).flatten()
print("Fraction model-scenario coverage:", len(np.where(~np.isnan(ar))[0])/len(ar))

Fraction model-scenario coverage: 0.925


## Now specify to only the c400 scenarios

In [29]:
ind_xr = xr.open_dataset(settings['paths']['data']+"XRindicators.nc").sel(Scenario=settings['scenarios_c400'], Model=list_of_models, Region='Europe')
data_xr = xr.open_dataset(settings['paths']['data']+"XRdata.nc").sel(Scenario=settings['scenarios_c400'], Model=list_of_models, Region='Europe')

## Plot

In [30]:
font_symbol = 18
font_axes = 30
font_annot = 25
names = ['B', 'C', 'N', 'V', 'El', 'H', 'F', 'Ef', '-']

In [40]:
fig = make_subplots(
    rows=3, cols=2,
    horizontal_spacing = 0.10,
    vertical_spacing=0.15,
    specs = [[{"rowspan": 2}, {"rowspan": 2}],
             [None, None],
             [{}, {}],
            ],
    subplot_titles = ("", "", "", ""),
)

for col_i, col in enumerate([1, 2]):
        var_x = [ind_xr.sel(Indicator="S3_speedmax", Time=2050).Value, ind_xr.sel(Indicator="M1_cir", Time=2050).Value][col_i]
        var_y = [ind_xr.sel(Indicator="S1_rai", Time=2050).Value, ind_xr.sel(Indicator="M2_eir", Time=2050).Value][col_i]

        # SHADED AREAS IN THE BACKGROUND
        ar_x = np.array(var_x)
        ar_y = np.array(var_y)
        ar_x = ar_x[ar_x != 0]
        ar_y = ar_y[ar_y != 0]
        m_y = np.nanmedian(ar_y)
        m_y1 = np.nanmedian(ar_y)+np.nanstd(ar_y)
        m_y1b = np.nanmedian(ar_y)+2*np.nanstd(ar_y)
        m_y0 = np.nanmedian(ar_y)-np.nanstd(ar_y)
        m_y0b = np.nanmedian(ar_y)-2*np.nanstd(ar_y)
        m_x = np.nanmedian(ar_x)
        m_x1 = np.nanmedian(ar_x)+np.nanstd(ar_x)
        m_x1b = np.nanmedian(ar_x)+2*np.nanstd(ar_x)
        m_x0 = np.nanmedian(ar_x)-np.nanstd(ar_x)
        m_x0b = np.nanmedian(ar_x)-2*np.nanstd(ar_x)
        mn_x = [105, 0.475][col_i]
        mn_y = [0.61, 0.36][col_i]

        fig.add_trace(go.Scatter(x=[0, mn_x, 1e3], y=[m_y]*3, text='\u03B7', textfont={"size": font_symbol}, mode='text+lines', showlegend=False, line={'dash':'dash'}, marker = {"symbol": "square", "color": 'silver', "size": 35, "line": {"color": "black", "width": 0}}), 1, col)
        fig.add_trace(go.Scatter(x=[0, mn_x, 1e3], y=[m_y1]*3, text='+\u03C3', textfont={"size": font_symbol}, mode='text', showlegend=False, marker = {"symbol": "square", "color": 'silver', "size": 35, "line": {"color": "black", "width": 0}}), 1, col)
        fig.add_trace(go.Scatter(x=[0, mn_x, 1e3], y=[m_y0]*3, text='-\u03C3', textfont={"size": font_symbol}, mode='text', showlegend=False, marker = {"symbol": "square", "color": 'silver', "size": 35, "line": {"color": "black", "width": 0}}), 1, col)
        fig.add_trace(go.Scatter(x=[0, mn_x, 1e3], y=[m_y1b]*3, text='+2\u03C3', textfont={"size": font_symbol}, mode='text', showlegend=False, marker = {"symbol": "square", "color": 'silver', "size": 35, "line": {"color": "black", "width": 0}}), 1, col)
        fig.add_trace(go.Scatter(x=[0, mn_x, 1e3], y=[m_y0b]*3, text='-2\u03C3', textfont={"size": font_symbol}, mode='text', showlegend=False, marker = {"symbol": "square", "color": 'silver', "size": 35, "line": {"color": "black", "width": 0}}), 1, col)
        fig.add_trace(go.Scatter(x=[-1e3, 1e3, 1e3, -1e3], y=[m_y0, m_y0, m_y1, m_y1], fillcolor='silver', line={'color':'silver'}, showlegend=False, fill='toself', opacity=0.25), 1, col)
        fig.add_trace(go.Scatter(x=[-1e3, 1e3, 1e3, -1e3], y=[m_y0b, m_y0b, m_y1b, m_y1b], showlegend=False, fillcolor='silver', line={'color':'silver'}, fill='toself', opacity=0.1), 1, col)

        fig.add_trace(go.Scatter(x=[m_x]*3, y=[-1e3, mn_y, 1e3], text='\u03B7', textfont={"size": font_symbol}, mode='text+lines', showlegend=False, line={'dash':'dash'}, marker = {"symbol": "square", "color": 'silver', "size": 35, "line": {"color": "black", "width": 0}}), 1, col)
        fig.add_trace(go.Scatter(x=[m_x1]*3, y=[-1e3, mn_y, 1e3], text='+\u03C3', textfont={"size": font_symbol}, mode='text', showlegend=False, line={'dash':'dash'}, marker = {"symbol": "square", "color": 'silver', "size": 35, "line": {"color": "black", "width": 0}}), 1, col)
        fig.add_trace(go.Scatter(x=[m_x0]*3, y=[-1e3, mn_y, 1e3], text='-\u03C3', textfont={"size": font_symbol}, mode='text', showlegend=False, line={'dash':'dash'}, marker = {"symbol": "square", "color": 'silver', "size": 35, "line": {"color": "black", "width": 0}}), 1, col)
        fig.add_trace(go.Scatter(x=[m_x1b]*3, y=[-1e3, mn_y, 1e3], text='+2\u03C3', textfont={"size": font_symbol}, mode='text', showlegend=False, line={'dash':'dash'}, marker = {"symbol": "square", "color": 'silver', "size": 35, "line": {"color": "black", "width": 0}}), 1, col)
        fig.add_trace(go.Scatter(x=[m_x0b]*3, y=[-1e3, mn_y, 1e3], text='-2\u03C3', textfont={"size": font_symbol}, mode='text', showlegend=False, line={'dash':'dash'}, marker = {"symbol": "square", "color": 'silver', "size": 35, "line": {"color": "black", "width": 0}}), 1, col)
        fig.add_trace(go.Scatter(x=[m_x0, m_x0, m_x1, m_x1], y=[-1e3, 1e3, 1e3, -1e3], fillcolor='silver', line={'color':'silver'}, showlegend=False, fill='toself', opacity=0.25), 1, col)
        fig.add_trace(go.Scatter(x=[m_x0b, m_x0b, m_x1b, m_x1b], y=[-1e3, 1e3, 1e3, -1e3], showlegend=False, fillcolor='silver', line={'color':'silver'}, fill='toself', opacity=0.1), 1, col)

        # ACTUAL DATA
        for m_i, m in enumerate(list_of_models):
                for s_i, s in enumerate(settings['scenarios_c400'][1:]):
                        fig.add_trace(go.Scatter(x=[var_x.sel(Scenario="DIAG-C400-lin", Model=m), var_x.sel(Scenario=s, Model=m)],
                                                y=[var_y.sel(Scenario="DIAG-C400-lin", Model=m), var_y.sel(Scenario=s, Model=m)], name=m,
                                                mode='lines+markers',
                                                showlegend=False,
                                                line={'width': 3, 'dash': 'solid'},
                                                text=["", names[s_i]],
                                                textfont = dict(color='black',
                                                                size=15),
                                                marker={"color": list_of_colors[m_i],
                                                        "size": [12, 30],
                                                        "line": {"color": "black", "width": 0.5}}
                                                ), 1, col)

        for m_i, m in enumerate(list_of_models):
                for s_i, s in enumerate(settings['scenarios_c400'][1:]):
                        fig.add_trace(go.Scatter(x=[var_x.sel(Scenario="DIAG-C400-lin", Model=m), var_x.sel(Scenario=s, Model=m)],
                                                y=[var_y.sel(Scenario="DIAG-C400-lin", Model=m), var_y.sel(Scenario=s, Model=m)], name=m,
                                                mode='text+markers',
                                                showlegend=False,
                                                text=["", names[s_i]],
                                                textfont = dict(color='black',
                                                                size=font_annot),
                                                marker={"color": list_of_colors[m_i],
                                                        "size": [12, 30],
                                                        "line": {"color": "black", "width": 0.5}}
                                                ), 1, col)

        # Dummy for legend
        for m_i, m in enumerate(list_of_models):
                fig.add_trace(go.Scatter(x=[-1e3],
                                        y=[1e3],
                                        mode='markers+text',
                                        showlegend=[True, False][col_i], line_width=4,
                                        name=list_of_model_shortnames[m_i],
                                        #text="<b>"+m[0]+"</b>",
                                        hovertemplate='<b>DIAG-C400-lin</b> <br>'+"ERT"+': %{y:.2f} <br>'+"Carbon capture"+': %{x:.2f}',
                                        marker={"color": list_of_colors[m_i],
                                                "size": 30,
                                                "symbol": "square",
                                                "line": {"color": "black", "width": 4}}
                                        ), 1, col)

        for m_i, m in enumerate(list_of_models):
                fig.add_trace(go.Scatter(x=[var_x.sel(Scenario="DIAG-C400-lin", Model=m)],
                                        y=[var_y.sel(Scenario="DIAG-C400-lin", Model=m)],
                                        mode='markers',
                                        showlegend=False, line_width=4,
                                        name=list_of_model_shortnames[m_i],
                                        #text="<b>"+m[0]+"</b>",
                                        text="<b>"+"-"+"</b>",
                                        hovertemplate='<b>DIAG-C400-lin</b> <br>'+"ERT"+': %{y:.2f} <br>'+"Carbon capture"+': %{x:.2f}',
                                        marker={"color": "black", #ModelCols_touse[m_i],
                                                "size": 12,
                                                "symbol": "circle",
                                                "line": {"color": "black", "width": 4}}
                                        ), 1, col)

        fig.update_xaxes(row=1, col=col, title=["Mitigation speed potential (R<sub>3</sub>)", 'Carbon Intensity Reduction (M<sub>1</sub>)'][col_i], title_font_size=font_axes, range=[[100, 310], [0.46, 1.3]][col_i])
        fig.update_yaxes(row=1, col=col, title=["Relative Abatement Index (R<sub>1</sub>)", 'Energy Intensity Reduction (M<sub>2</sub>)'][col_i], title_font_size=font_axes, range=[[0.60, 1.3], [0.35, 0.70]][col_i])


for s_i, s in enumerate(settings['scenarios_c400'][1:]+['DIAG-C400-lin']):
        fig.add_trace(go.Scatter(x=[260],
                                y=[1.25-s_i*0.03], name=m,
                                mode='text+markers',
                                showlegend=False,
                                text=[names[s_i]],
                                textfont = dict(color='black',
                                                size=font_annot-10),
                                marker={"color": 'white',
                                        "size": 25,
                                        "line": {"color": "black", "width": 0.5}}
                                ), 1, 1)
        strin = s[14:]
        if s == "DIAG-C400-lin": strin = "DIAG-C400-lin"
        fig.add_annotation(x=260+25, y=1.25-s_i*0.03, text=strin, showarrow=False, align='left', font=dict(size=14))


for m_i, m in enumerate(list_of_models):
        fig.add_trace(go.Scatter(x=ind_xr.Time,
                                y=ind_xr.sel(Indicator = "S4_sensprim", Model=m, Scenario='DIAG-C400-lin').Value,
                                mode='lines',
                                showlegend=False, line_width=4,
                                name=list_of_model_shortnames[m_i],
                                line = {'color': list_of_colors[m_i]}
                                ), 3, 1)
fig.update_xaxes(row=3, col=1, title="Time (years)", title_font_size=font_axes, range=[2020, 2100])
fig.update_yaxes(row=3, col=1, type="log", title="Sensitivity<br>primary energy (S<sub>4</sub>)", title_font_size=font_axes-5, range=[-4, 1])

for m_i, m in enumerate(list_of_models):
        fig.add_trace(go.Scatter(x=ind_xr.Time,
                                y=ind_xr.sel(Indicator = "S5_sensdem", Model=m, Scenario='DIAG-C400-lin').Value,
                                mode='lines',
                                showlegend=False, line_width=4,
                                name=list_of_model_shortnames[m_i],
                                line = {'color': list_of_colors[m_i]}
                                ), 3, 2)
fig.update_xaxes(row=3, col=2, title="Time (years)", title_font_size=font_axes, range=[2020, 2100])
fig.update_yaxes(row=3, col=2, type="log", title="Sensitivity<br>demand (S<sub>5</sub>)", title_font_size=font_axes-5, range=[-6, -1])

# Panel annotations
fig.add_annotation(x=107, y=1.28, text="(a)", showarrow=False, font=dict(size=30), row=1, col=1)
fig.add_annotation(x=0.49, y=0.69, text="(b)", showarrow=False, font=dict(size=30), row=1, col=2)
fig.add_annotation(x=2023, y=0.5, text="(c)", showarrow=False, font=dict(size=30), row=3, col=1)
fig.add_annotation(x=2023, y=-1.5, text="(d)", showarrow=False, font=dict(size=30), row=3, col=2)

#fig.update_annotations(font_size=font_annot)
fig.update_layout(font_size = 20)
fig.update_layout(legend=dict(x=0.04, y=1.1, orientation="h", font_size=18))
fig.update_layout(height=1200, width=1800)
fig.update_layout(template="simple_white")
fig.show()
fig.write_image(settings['paths']['figures'] + "Figure_2.png", scale=4)